In [ ]:
#internet parsing
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

URL = 'https://code.s3.yandex.net/data-analyst-eng/chicago_weather_2017.html'
req = requests.get(URL)
#req.text
soup=BeautifulSoup(req.text, 'lxml')

heading_table = [] # List where the names of the columns will be stored
for row in soup.find_all('th'): # The names of the columns are inside <th> elements, 
# so we'll find all <th> elements in the table and run them through in a loop
        heading_table.append(row.text) # Add content from the <th> tag to heading_table list using append() 
#print(heading_table)

content=[] # list where the table data will be stored
for row in soup.find_all('tr'): 
# Each row is wrapped in a <tr> tag, we need to loop through all the rows
    if not row.find_all('th'): 
# We need this condition to ignore the first row of the table, with headings
            content.append([element.text for element in row.find_all('td')])
            # Within each row the cell content is wrapped in <td> </td> tags
            # We need to loop through all <td> elements, extract the content from the cells, 
            # and add it to the list
            # Then add each of the lists to the content list
#print(content)

weather_records = pd.DataFrame(content, columns=heading_table) 
# pass two-dimensional content list as data and heading_table as headings
print(weather_records)

1.
Print the company_name field. Find the number of taxi rides for each taxi company for November 15-16, 2017, name the resulting field trips_amount, and print it, too. Sort the results by the trips_amount field in descending order.
Hint
Join the tables cabs and trips. Use aggregate functions and grouping. Don't forget to introduce a condition.

In [ ]:
SELECT
    cabs.company_name,   
    COUNT(trips.trip_id) AS trips_amount
FROM
    trips
INNER JOIN cabs on cabs.cab_id = trips.cab_id
WHERE
    CAST(trips.start_ts AS date) between '2017-11-15' AND '2017-11-16'    
GROUP BY
    cabs.company_name
ORDER BY
    trips_amount DESC;

2.
Find the number of rides for every taxi company whose name contains the words "Yellow" or "Blue" for November 1-7, 2017. Name the resulting variable trips_amount. Group the results by the company_name field.
Hint
Join the tables cabs and trips. You can make calculations for one group of companies (e.g. those with the word "Yellow" in their name), then for the other, and finally group the results.

In [ ]:
SELECT
    cabs.company_name,   
    COUNT(trips.trip_id) AS trips_amount
FROM
    trips
INNER JOIN cabs on cabs.cab_id = trips.cab_id
WHERE
    CAST(trips.start_ts AS date) between '2017-11-01' AND '2017-11-07'
    AND (cabs.company_name LIKE '%Yellow%' OR cabs.company_name LIKE '%Blue%')
GROUP BY
    cabs.company_name
ORDER BY
    trips_amount DESC;

You correctly completed the task

3.
For November 1-7, 2017, the most popular taxi companies were Flash Cab and Taxi Affiliation Services. Find the number of rides for these two companies and name the resulting variable trips_amount. Join the rides for all other companies in the group "Other." Group the data by taxi company names. Name the field with taxi company names company. Sort the result in descending order by trips_amount.
Hint
Join the tables cabs and trips. Use the CASE construction to group the companies.

In [ ]:
SELECT
        
CASE 
	WHEN (cabs.company_name = 'Flash Cab' OR cabs.company_name = 'Taxi Affiliation Services') 
    THEN cabs.company_name --'trips_amount'
    ELSE  'Other'
END AS company,
    COUNT(trips.trip_id) AS trips_amount
FROM
    trips
INNER JOIN cabs on cabs.cab_id = trips.cab_id
WHERE
    CAST(trips.start_ts AS date) between '2017-11-01' AND '2017-11-07'
    --AND (cabs.company_name = 'Flash Cab' OR cabs.company_name = 'Taxi Affiliation Services')
GROUP BY
    company
ORDER BY
    trips_amount DESC;

4.
Find out where rides most often end. Find the average daily number of drop-offs in each Chicago neighborhood in the month that we have data for (November 2017). Name the resulting field average_trips. To retrieve the names of the neighborhoods, use the name field from the neighborhoods table. Rename it as dropoff_location_name in the resulting table. Sort the result in descending order by average_trips.
Hint
Join the tables trips and neighborhoods. In the subquery, get records for each day for each neighborhood. Use the EXTRACT operator (day from trips.end_ts) to retrieve data for individual days. Then take the average.

In [ ]:
SELECT
    neighborhoods.name AS dropoff_location_name,
    AVG(SUBQ.countrides)AS average_trips
    
FROM neighborhoods
INNER JOIN
    (SELECT
     COUNT(trips.dropoff_location_id) AS countrides,
     dropoff_location_id,
     EXTRACT(DAY FROM CAST(trips.end_ts AS date)) AS day
     FROM trips
     GROUP BY day, dropoff_location_id
     ) AS SUBQ ON SUBQ.dropoff_location_id = neighborhoods.neighborhood_id
GROUP BY
    dropoff_location_name
ORDER BY
    average_trips DESC;

5.
Retrieve the identifiers of the O'Hare and Loop neighborhoods from the neighborhoods table.
Hint
Use the condition name LIKE '%Hare' OR name LIKE 'Loop'
The result should look like this:
    
neighborhood_id	name	Column
50	Loop	
63	O'Hare	

In [ ]:
SELECT
   *
FROM
    neighborhoods
WHERE name LIKE '%Hare' OR name LIKE 'Loop'
    ;

6.
For each hour, retrieve the weather condition records from the weather_records table. Using the CASE operator, break all hours into two groups: "Bad" if the description field contains the words '"rain" or "storm," and "Good" for others. Name the resulting field weather_conditions. The final table must include two fields: date and hour (ts) and weather_conditions.
Hint
Use the operators CASE, LIKE, and OR.

In [ ]:
SELECT
--ts :: date AS date
--EXTRACT(hour FROM ts) AS hour,

ts,
CASE
    WHEN description LIKE '%rain%' OR description LIKE '%storm%'
    THEN 'Bad'
    ELSE 'Good'
END AS weather_conditions

FROM weather_records;

7.
Retrieve from the trips table all the rides that started in the Loop (neighborhood_id: 50) and ended at O'Hare (neighborhood_id: 63) on a Saturday. Get the weather conditions for each ride. Use the method you applied in the previous task. Also retrieve the duration of each ride. Ignore rides for which data on weather conditions is not available.
The table columns should be in the following order:

    start_ts
    weather_conditions
    duration_seconds

Sort by start_ts.

Hint
Use INNER JOIN and apply EXTRACT (DOW from trips.start_ts) = 6 to retrieve data on Saturday rides.

In [ ]:
SELECT
   trips.start_ts,
   
CASE
    WHEN description LIKE '%rain%' OR description LIKE '%storm%'
    THEN 'Bad'
    ELSE 'Good'
END AS weather_conditions,
 trips.duration_seconds
    
FROM
    trips
INNER JOIN weather_records ON weather_records.ts = trips.start_ts
WHERE
pickup_location_id = 50 -- start at Loop
AND
dropoff_location_id = 63
AND EXTRACT(dow FROM start_ts) = 6 -- Saturday = 6

ORDER BY
start_ts;

## You sure have learned a lot about SQL. Congratulations!